In [71]:
from gensim import models
from random import shuffle
from nltk.tokenize import word_tokenize
import json
import multiprocessing
from gensim.models.doc2vec import Doc2Vec, TaggedDocument
import gensim.models.doc2vec
import numpy as np
import gzip
import time
import subprocess
import random
import arff
from subprocess import Popen, PIPE, STDOUT

In [230]:
data = []
with gzip.open('/data/shared/reviews/amazon_snap/reviews_Video_Games_5.json.gz') as f:
    for line in f:
        data.append(json.loads(line))

print('Total reviews in dataset: ', len(data))

Total reviews in dataset:  231780


In [ ]:
#If there is no tokenizer, download here

#import nltk
#nltk.download()
len(data)

In [298]:
sentences = []

for l in range(len(data)):
    sentence = models.doc2vec.LabeledSentence(
        words = word_tokenize(data[l]['reviewText'].lower()), 
        tags = [data[l]['reviewerID']+'|'+data[l]['asin']])
    if len(sentence[0]) > 25:
        sentences.append(sentence)

print('Total reviews introduced: ', len(sentences))

Total reviews introduced:  217147


In [313]:
assert gensim.models.doc2vec.FAST_VERSION > -1, "this will be painfully slow otherwise"

model_dm = models.Doc2Vec(dm=1, dm_mean=1, size=100, window=10, min_count=3, workers=multiprocessing.cpu_count(), alpha=.025, min_alpha=.025)
#model.build_vocab(sentences)

model_dbow = models.Doc2Vec(dm=0, dbow_words=1, size=100, window=10, min_count=3, workers=multiprocessing.cpu_count())

In [314]:
model_dbow.build_vocab(sentences)
print(str(model_dbow))

model_dm.build_vocab(sentences)
print(str(model_dm))

Doc2Vec(dbow+w,d100,n5,w10,mc3,s0.001,t4)
Doc2Vec(dm/m,d100,n5,w10,mc3,s0.001,t4)


In [315]:
assert len(model_dbow.docvecs) == len(sentences), "there are overlapping section titles! {0} docvecs and {1} documents".format(len(model.docvecs), len(sentences))
assert len(model_dm.docvecs) == len(sentences), "there are overlapping section titles! {0} docvecs and {1} documents".format(len(model.docvecs), len(sentences))

In [ ]:
for epoch in range(10):
    print("Training doc2vec_dbow model, epoch {}".format(epoch)) 
    #Randomly shuffle sentences for better training
    #shuffle(sentences)
    model_dbow.train(sentences)
    #model_dbow.alpha -= 0.002 # decrease the learning rate
    model_dbow.min_alpha = model_dbow.alpha # fix the learning rate, no decay
    
model_dbow.save("/data/shared/trainedmodels/VidGam_dbow10epoch25.doc2vec")

Training doc2vec_dbow model, epoch 0
Training doc2vec_dbow model, epoch 1
Training doc2vec_dbow model, epoch 2
Training doc2vec_dbow model, epoch 3
Training doc2vec_dbow model, epoch 4
Training doc2vec_dbow model, epoch 5
Training doc2vec_dbow model, epoch 6


In [319]:
test_dbow = model_dbow.docvecs.similarity( d1='A34UVV757IKPVB|B004FSE52C', d2='A34UVV757IKPVB|B004FSE52C')
test_dbow

1.0

In [4]:
model_dbow = gensim.models.Doc2Vec.load("/data/shared/trainedmodels/VidGam_dbow10epoch.doc2vec")

In [258]:
inverted_sentences = {}

for x in range(len(sentences)):
    a = sentences[x][1][0]
    b = ' '.join(sentences[x][0])
    inverted_sentences[a] = b

In [317]:
# Cool example: A17NVM7IAPF2NS|B002I08RR8
r = np.random.randint(len(sentences))

dbow_sim = model_dbow.docvecs.most_similar(positive=sentences[r][1], topn=5)
print('Top 5 similar reviews in dbow to: ', inverted_sentences[sentences[r][1][0]]+'\n')
for x in range(len(dbow_sim)):
    print('ID: ', dbow_sim[x][0], ' Review: ',inverted_sentences[dbow_sim[x][0]]+'\n')

Top 5 similar reviews in dbow to:  first things first -if you have n't played the first assassin 's creed and subsequent releases ( or at least starting with ii , and going forward ) - go no further . do yourself a service and start with ii as a bare minimum and work yourself forward to this , the 3rd in the ezio auditore trilogy . otherwise , you 'll be completely lost.for those of you familiar with the story , you 're going to find ezio auditore in his twilight years on a quest to find altair 's legendary , yet quite well-protected , library . the templars have caught wind of it and ezio is also trying to find his way in to thwart their attempts at worldwide control . this is hardly a spoiler as you 're quickly brought into this within minutes of beginning to play ezio - no further spoilers here.from the standpoint of playing all of the previous assassin 's creed entries , how does this one fare ? well , i finally played it tonight and i must say that it is really quite remarkable . 

In [329]:
def product_similars_model(review_id, number):    
    dbow_sim = model_dbow.docvecs.most_similar(positive=[review_id], topn=10000)
    print('Top', number, 'similar reviews in dbow to: ', inverted_sentences[review_id]+'\n')
    i = 0
    for x in range(len(dbow_sim)):
        s = dbow_sim[x][0]
        values = s[(s.index('|'))+1:]
        if i < number+1:
            if values == review_id[(review_id.index('|'))+1:]:
                print('ID: ', dbow_sim[x][0], ' Review: ',inverted_sentences[dbow_sim[x][0]]+'\n')
                i = i + 1

#dbow_sim = model_dbow.docvecs.most_similar(positive=['A1XCEM1Q832SCG|B0050SYX8W'], topn=400)
#print('Top 5 similar reviews in dbow to: ', inverted_sentences['A1XCEM1Q832SCG|B0050SYX8W']+'\n')
#i = 0
#for x in range(len(dbow_sim)):
#    s = dbow_sim[x][0]
#    print(s)
#    value = s[(s.index('|'))+1:]
#    if i < 11:
#        if value == 'B0050SYX8W':
#            print('ID: ', dbow_sim[x][0], ' Review: ',inverted_sentences[dbow_sim[x][0]]+'\n')
#            i = i + 1

In [331]:
product_similars_model('AFNG8O2DXRCUV|B0050SYX8W', 5)
product_similars_model('A1XCEM1Q832SCG|B0050SYX8W', 5)
product_similars_model('APA6WLVT0ZOL1|B0050SYX8W', 5)

Top 5 similar reviews in dbow to:  while our family plays kinect games , i personally play action / role-playing games on ps3 . i loved the ps3 exclusives like uncharted , killzone , resistance , but also loved cross-platform games like cod series , battlefield , and especially mass effect 2 and 3. i love the strong stories of mass effect balanced with battle actions , and this is where it leads me to try halo 4.i like the environment in halo 4 very much - space and fantasy worlds . the details are on par with the very best . in the space ship , details including steams / smoke , electrical sparks are all very realistic . for the nature scenes , many of the tree , rock , and plant details are well articulated . animation / frame rate is very smooth . i experienced no lag . the details on the faces of humans are among the best there is . i do , however , feel many of the ship and platform structures and corridors are very similar . this leads me to believe it 's the capacity limitation 

In [312]:
for x in range(len(data)):
    if data[x]['reviewerID'] == 'A3APAEKRHT2QH4':
        print(data[x])
#sentences[1000][1]
#halo_4[0][1]#['A262YOBA44YU4M|B0050SYX8W']
#len(model_dbow)
data[0]#['asin']

{'reviewerID': 'A3APAEKRHT2QH4', 'asin': 'B0029LJIFG', 'reviewerName': 'Charity Bridenbecker', 'helpful': [0, 0], 'reviewText': 'yeah this was great I got my xbox gold for a year and did not have to leave home works for me no time to go to the store to busy with work school kids and life.', 'overall': 5.0, 'summary': ':)', 'unixReviewTime': 1378857600, 'reviewTime': '09 11, 2013'}
{'reviewerID': 'A3APAEKRHT2QH4', 'asin': 'B0037LTTRO', 'reviewerName': 'Charity Bridenbecker', 'helpful': [0, 0], 'reviewText': 'another game that my man enjoys play so much he loves the fact that I got it for him. He looked all over and could not find it he was shocked when he got it for christmas', 'overall': 5.0, 'summary': 'made my man happy', 'unixReviewTime': 1378857600, 'reviewTime': '09 11, 2013'}
{'reviewerID': 'A3APAEKRHT2QH4', 'asin': 'B003O6CA64', 'reviewerName': 'Charity Bridenbecker', 'helpful': [0, 1], 'reviewText': 'this was one of the hard to find games that my man looked for for a long time 

{'asin': '0700099867',
 'helpful': [8, 12],
 'overall': 1.0,
 'reviewText': 'Installing the game was a struggle (because of games for windows live bugs).Some championship races and cars can only be "unlocked" by buying them as an addon to the game. I paid nearly 30 dollars when the game was new. I don\'t like the idea that I have to keep paying to keep playing.I noticed no improvement in the physics or graphics compared to Dirt 2.I tossed it in the garbage and vowed never to buy another codemasters game. I\'m really tired of arcade style rally/racing games anyway.I\'ll continue to get my fix from Richard Burns Rally, and you should to. :)http://www.amazon.com/Richard-Burns-Rally-PC/dp/B000C97156/ref=sr_1_1?ie=UTF8&qid;=1341886844&sr;=8-1&keywords;=richard+burns+rallyThank you for reading my review! If you enjoyed it, be sure to rate it as helpful.',
 'reviewTime': '07 9, 2012',
 'reviewerID': 'A2HD75EMZR8QLN',
 'reviewerName': '123',
 'summary': "Pay to unlock content? I don't think so

In [6]:
import arff

In [7]:
def vector_arff(labeled_sentences, model, name):
    """
    (LabeledSentences, doc2vec model, string) -> dict
    
    Requirements: 
    -All vectors are the same size
    
    Returns a dictionary ready for arff encoding.
    
    """
    
    vectors_arff = {}
    attributes = []
    data_arff = []
    
    for attribute in range(len(model.docvecs[sentences[0][1][0]])):
        attributes.append(tuple(('dim_{}'.format(str(attribute)), 'REAL')))
        
    for x in range(len(sentences)-1):
        vec = []
        vec = model.docvecs[sentences[x][1][0]]
        vec_list = vec.tolist()
        data_arff.append(vec_list)
    
    vectors_arff['relation'] = name
    vectors_arff['attributes'] = attributes
    vectors_arff['data'] = data_arff
    
    return vectors_arff
    print('Done')

In [ ]:
#pet_arff = vector_arff(sentences, model_dbow, 'pet_amazon')

#arff.dump(vector_arff(sentences_10K, model_dbow, 'videogames_amazon'), open('VidGam_rev_10K.arff', 'w'))

VidGam_rev_arff_10K = arff.load(open('/data/work/manuel/OpenSubspace/data/VidGam_rev_10K.arff', 'r'))

The code to run the Subspace Clustering algorithm from here to the command line looks like this:

<code>subprocess.check_output(['java', '-Xmx:memory', '-cp', 'jar location', 'algorithm', -t', 'arff file', 'cluster args'])</code>

Some algorithm examples, with parameters and defaults:

* *PROCLUS* -K (4), -D (3): In PROCLUS, K and D are pretty self-explanatory, they mean the amount of clusters to find, as well as the average dimentionality of the clusters. It's a fairly quick algorithm, but it requires some knowledge or restriction of the clusters needed.

* *CLIQUE* -XI (10), -TAU (1.0): CLIQUE defines a cluster as a connection of grid cells with each more than τ (TAU) many objects. Grid cells are defined by a fixed grid splitting each dimension in ξ (XI) equal width cells.

* *MINECLUS* -a (0.08), -b (0.25), -w (0.2), -m (-1), -n (1), -k (5): MINECLUS is an optimization of DOC by using Frequent Pattern trees to improve decision time and accuracy, which is an optimization of CLIQUE itself, by using flexible hypercubes of width _W_ instead of a fixed grid. α (0,1] is the minimun density of the discovered clusters, β (0,1] is a balance condition between the number of points and number of dimensions in a cluster, the measure  M is the MAXOUT parameter, n is the number of bins and k ???

To work with several parameters, we can run experiments specifying the algorithm and tuning the values as necessary. It may not be intuitive and small changes can mean a large difference.

In [ ]:
import time
start = time.time()
proclus_S1500 = subprocess.check_output(['java', '-Xmx2048m', '-cp', '/data/work/manuel/OpenSubspace/*', 'weka/subspaceClusterer/Proclus',
                '-t', '/data/work/manuel/OpenSubspace/data/S1500.arff', '-K', '5', '-D', '10' ]).decode()
end = time.time()
print('Done Proclus in', (end-start)//60 , ' min')

start = time.time()
clique_S1500 = subprocess.check_output(['java', '-Xmx2048m', '-cp', '/data/work/manuel/OpenSubspace/*', 'weka/subspaceClusterer/Clique',
                '-t', '/data/work/manuel/OpenSubspace/data/S1500.arff' ]).decode()
end = time.time()
print('Done Clique in', (end-start)//60 , ' min')

start = time.time()
doc_S1500 = subprocess.check_output(['java', '-Xmx2048m', '-cp', '/data/work/manuel/OpenSubspace/*', 'weka/subspaceClusterer/Doc',
                '-t', '/data/work/manuel/OpenSubspace/data/S1500.arff' ]).decode()
end = time.time()
print('Done Doc in', (end-start)//60 , ' min')

In [ ]:
#print(proclus_experiments['proclus_S1500_k3_d5'][4])

In [ ]:
XI = [10, 5]
TAU = [0.5, 0.49]
clique_exp_vidgam = {}
algorithm = 'Clique'

for i in XI:
    for j in TAU:
        start = time.time()
#        clique_experiments["clique_pet_XI{0}_TAU{1}".format(i,j)] = subprocess.check_output(['java', '-Xmx4096m', '-cp', 
#                                                                    '/data/work/manuel/OpenSubspace/*', 
#                                                                    'weka/subspaceClusterer/{0}'.format(algorithm),
#                                                                    '-t', '/data/work/manuel/OpenSubspace/data/pet_rev.arff', 
#                                                                    '-XI', str(i), '-TAU', str(j) ]).decode()
        results = []
        p = Popen(['java', '-Xmx4096m', '-cp', '/data/work/manuel/OpenSubspace/*', 'weka/subspaceClusterer/{0}'.format(algorithm),
                   '-t', '/data/work/manuel/OpenSubspace/data/VidGam_rev.arff', '-XI', str(i), '-TAU', str(j) ], 
                  stdout=PIPE, stderr=STDOUT)
        for line in p.stdout:
            results.append(line.decode())
        clique_exp_vidgam["clique_VidGam_XI{0}_TAU{1}".format(i,j)] = results
        end = time.time()
        print('Done {0}_k{1}_d{2}'.format(algorithm,i,j), 'with XI=', i, ', TAU=', j, ' in ', (end-start), ' sec')

In [ ]:
ALPHA = [0.001, 0.1, 0.5]
BETA = [0.1, 0.4]
MAXOUT = [-1, 1]
K = [2] #up to 64?
numBins = [0.1, 1, 10]
W = [0.5, 50, 100]

mineclus_exp_pet = {}
algorithm = 'MineClus'

for i in ALPHA:
    for j in BETA:
        for k in MAXOUT:
            for l in K:
                for m in numBins:
                    for n in W:
                        start = time.time()
                #        clique_experiments["clique_pet_XI{0}_TAU{1}".format(i,j)] = subprocess.check_output(['java', '-Xmx4096m', '-cp', 
                #                                                                    '/data/work/manuel/OpenSubspace/*', 
                #                                                                    'weka/subspaceClusterer/{0}'.format(algorithm),
                #                                                                    '-t', '/data/work/manuel/OpenSubspace/data/pet_rev.arff', 
                #                                                                    '-XI', str(i), '-TAU', str(j) ]).decode()
                        results = []
                        p = Popen(['java', '-Xmx4096m', '-cp', '/data/work/manuel/OpenSubspace/*', 'weka/subspaceClusterer/{0}'.format(algorithm),
                                   '-t', '/data/work/manuel/OpenSubspace/data/pet_rev.arff', '-a', str(i), '-b', str(j), '-m', str(k), 
                                   '-k', str(l), '-n', str(m), '-w', str(n)], 
                                  stdout=PIPE, stderr=STDOUT)
                        for line in p.stdout:
                            results.append(line.decode())
                        mineclus_exp_pet["mineclus_pet_{}_{}_{}_{}_{}_{}".format(i,j,k,l,m,n)] = results
                        end = time.time()
                        print('Done {}_{}_{}_{}_{}_{}_{}'.format(algorithm,i,j,k,l,m,n), ' in ', (end-start), ' sec')

In [35]:
def clusters_detail(experiment):
    print(experiment[0], 'Number of Clusters', len(experiment)-3)
    for i in range(len(experiment)):
        s = experiment[i]
        if '[' in s:
            print('# instances in cluster {}:'.format(i-2), s[(s.index(']')+3):(s.index('{'))])
            print('dimensions of clusters:', s[(s.index('[')):(s.index(']')+1)])
    print('-'*100)
    
def clusters_info(experiment):
    print(experiment[0], 'Number of Clusters', len(experiment)-3)
    dims = []
    for i in range(len(experiment)):
        s = experiment[i]
        if '[' in s:
            #print('# instances in cluster {}:'.format(i-2), s[(s.index(']')+3):(s.index('{'))])
            dims.append(int((s[(s.index('[')):(s.index(']')+1)]).count('1')))
            #dims.append('1')
            #print(int((s[(s.index('[')):(s.index(']')+1)]).count('1')))
    if dims: print('Average dimensions =', str(round(np.mean(dims), 2)))   
    #print('dims=',dims)
    print('-'*100)

In [ ]:
for key in proclus_pet: clusters_detail(proclus_pet[key])

#for key in clique_exp_vidgam: print(key)#clique_exp_vidgam[key])

#clusters_in_exp(clique_exp_vidgam['clique_VidGam_XI5_TAU0.49'])


#for key in proclus_experiments: print(proclus_experiments[key])
#len(proclus_experiments['proclus_S1500_k3_d5'])
#for i in range(len(proclus_experiments['proclus_S1500_k3_d5'])):
#for key in clique_experiments: print(len(clique_experiments[key]))
#3clique_experiments['clique_pet_XI10_TAU0.6'] 
#clique_experiments['clique_pet_Xi']#[2]#[200:250]

In [ ]:
file = 'VidGam_rev.arff'
k = [6]
d = [20, 50, 90]
proclus_vg = {}
Proclus_Experiment(k, d, proclus_vg, file)

In [83]:
def Proclus_Experiment(clusters, dimensions, dictionary, dataset):
    algorithm = 'Proclus'    
    from subprocess import Popen, PIPE, STDOUT
    for i in clusters:
        for j in dimensions:
            start = time.time()
            results = []
            p = Popen(['java', '-Xmx6g', '-cp', '/data/work/manuel/OpenSubspace/*', 
                       'weka/subspaceClusterer/{0}'.format(algorithm),
                       '-t', '/data/work/manuel/OpenSubspace/data/{}'.format(dataset), 
                       '-K', str(i), '-D', str(j) ], 
                       stdout=PIPE, stderr=STDOUT)
            for line in p.stdout:
                results.append(line.decode())
            dictionary["exp_k{0}_d{1}".format(i,j)] = results
            end = time.time()
            print('Done {0}_k{1}_d{2}'.format(algorithm,i,j), 'with clusters=', i, 
                  ', dimensions=', j, ' in ', (end-start), ' sec')

def Mineclus_Experiment(ALPHA, BETA, MAXOUT, K, numBins, W, dictionary, dataset):    
    algorithm = 'MineClus'
    for i in ALPHA:
        for j in BETA:
            for k in MAXOUT:
                for l in K:
                    for m in numBins:
                        for n in W:
                            start = time.time()
                            results = []
                            p = Popen(['java', '-Xmx6g', '-cp', '/data/work/manuel/OpenSubspace/*', 
                                       'weka/subspaceClusterer/{0}'.format(algorithm),
                                       '-t', '/data/work/manuel/OpenSubspace/data/{}'.format(dataset), 
                                       '-a', str(i), '-b', str(j), '-m', str(k), 
                                       '-k', str(l), '-n', str(m), '-w', str(n)], 
                                      stdout=PIPE, stderr=STDOUT)
                            for line in p.stdout:
                                results.append(line.decode())
                            dictionary["exp_a{}_b{}_m{}_k{}_n{}_w{}".format(i,j,k,l,m,n)] = results
                            end = time.time()
                            print('Done {}_a{}_b{}_m{}_k{}_n{}_w{}'.format(algorithm,i,j,k,l,m,n), ' in ', (end-start), ' sec')

def sample_from_cluster(experiment, cluster, size):
    s = experiment[cluster+2]
    values = s[(s.index('{'))+1:(s.index('}')-1)].split(' ')
    sample = []
    if size > 0:
        for i in range(size): sample.append(int(random.choice(values)))
    else:
        for i in range(10): sample.append(int(random.choice(values)))
    return sample

def sample_sentences_from_cluster(experiment, cluster, size):
    s = experiment[cluster+2]
    values = s[(s.index('{'))+1:(s.index('}')-1)].split(' ')
    sample = []
    if size > 0:
        for i in range(size): 
            rdm = int(random.choice(values))
            sample.append(['Sentence #{}:'.format(i+1), sentences[rdm][1][0], ' '.join(sentences[rdm][0])])
    else:
        for i in range(10): sample.append('Sentence #{}:'.format(i+1) ,' '.join(sentences[int(random.choice(values))][0]))
    return sample

In [ ]:
# Verify that arff index is equal to the sentences index, and the review they refer to:
print(model_dbow.docvecs[sentences[1][1][0]])
print(VidGam_rev_arff['data'][1])
sentences[1][0]

# Explore details of the clusters
#for key in proclus_vg: clusters_detail(proclus_vg[key])
clusters_detail(proclus_vg['exp_k6_d50'])

In [ ]:
# Check the list of samples, and the sentences
sample_proclusk6d50 = sample_from_cluster(proclus_vg['exp_k6_d50'], 0, 10)

samples_proclusk6d50 = {}
for i in range(6):
    samples_proclusk6d50['cluster_{}'.format(i)] = sample_sentences_from_cluster(proclus_vg['exp_k6_d50'], i, 10)
    
sample_proclusk6d50_0 = sample_sentences_from_cluster(proclus_vg['exp_k6_d50'], 0, 10)
sample_proclusk6d50_1 = sample_sentences_from_cluster(proclus_vg['exp_k6_d50'], 1, 10)

In [ ]:
ALPHA = [0.01]#, 0.1]
BETA = [0.2]#, 0.4]
MAXOUT = [1]
K = [20] #up to 64?
numBins = [1]
W = [1.5]
mineclus_vg10K = {}
Mineclus_Experiment(ALPHA, BETA, MAXOUT, K, numBins, W, mineclus_vg10K, 'VidGam_rev_10K.arff')

In [ ]:
sample_proclusk6d50_1



In [ ]:
samples_proclusk6d50['cluster_1']

In [ ]:
with open('cluster_samples.json', 'w') as fp:
    json.dump(samples_proclusk6d50, fp)

with open('cluster_samples.json') as cluster_samples:    
    data = json.load(cluster_samples)

In [ ]:
from pprint import pprint

#pprint(data)

In [267]:
from collections import defaultdict

s = sentences
d = defaultdict(list)

for i in range(len(sentences)):
    s = sentences[i][1][0]
    asin = s[(s.index('|')+1):]
    d[asin].append(i)

for key in d:
    if len(d[key]) > 350:
        print(key)

B0009VXBAQ
B0015AARJI
B00178630A
B002VBWIP6
B0050SYX8W
B007CM0K86
B00BGA9WK2


* B0009VXBAQ - Nintendo Wii
* B0015AARJI - PS3 Controller
* B00178630A - Diablo III
* B002VBWIP6 - 
* B0050SYX8W - Halo 4

In [45]:
halo_4 = []
for id in d['B0050SYX8W']:
    halo_4.append(sentences[id])

In [50]:
halo_4[400] == sentences[175850]
#len(halo_4)

True

In [202]:
def vector_arff_sub(labeled_sentences, model, name, start, stop):
    """
    (LabeledSentences, doc2vec model, string) -> dict
    
    Requirements: 
    -All vectors are the same size
    
    Returns a dictionary ready for arff encoding.
    
    """
    
    vectors_arff = {}
    attributes = []
    data_arff = []
    
    for attribute in range(len(model.docvecs[sentences[0][1][0]])):
        attributes.append(tuple(('dim_{}'.format(str(attribute)), 'REAL')))
        
    for x in range(start, stop):
        vec = []
        vec = model.docvecs[sentences[x][1][0]]
        vec_list = vec.tolist()
        data_arff.append(vec_list)
    
    vectors_arff['relation'] = name
    vectors_arff['attributes'] = attributes
    vectors_arff['data'] = data_arff
    
    return vectors_arff
    print('Done')
    
def sample_sentences_from_cluster_sub(experiment, cluster, size, sentences):
    s = experiment[cluster]
    values = s[(s.index('{'))+1:(s.index('}')-1)].split(' ')
    sample = []
    random_list = random.sample(values, size)
    if size > 0:
        for x in random_list:
            sample.append(['Sentence #{}:'.format(x), sentences[int(x)][1][0], ' '.join(sentences[int(x)][0])])
    
    #if size > 0:
    #    for i in range(size): 
    #        rdm = int(random.choice(values))
    #        sample.append(['Sentence #{}:'.format(i+1), sentences[rdm][1][0], ' '.join(sentences[rdm][0])])
    #else:
    #    for i in range(10): sample.append('Sentence #{}:'.format(i+1) ,' '.join(sentences[int(random.choice(values))][0]))
    return sample

In [51]:
#arff.dump(vector_arff(sentences_10K, model_dbow, 'videogames_amazon'), open('VidGam_rev_10K.arff', 'w'))

arff.dump(vector_arff_sub(sentences, model_dbow, 'halo_4', 175450, 175850), open('halo_4.arff', 'w'))

#halo_4_arff = arff.load(open('/data/work/manuel/OpenSubspace/data/halo_4.arff', 'r'))

<_io.TextIOWrapper name='halo_4.arff' mode='w' encoding='UTF-8'>

In [210]:
k = [4]
d = [50, 75, 95]
proclus_h4 = {}
Proclus_Experiment(k, d, proclus_h4, 'halo_4.arff')

Done Proclus_k4_d50 with clusters= 4 , dimensions= 50  in  0.5199093818664551  sec
Done Proclus_k4_d75 with clusters= 4 , dimensions= 75  in  0.6405737400054932  sec
Done Proclus_k4_d95 with clusters= 4 , dimensions= 95  in  0.7482900619506836  sec


In [42]:
print(model_dbow.docvecs[halo_4[1][1][0]] == halo_4_arff['data'][1])
len(halo_4_arff['data'])

[ True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True  True  True  True  True  True  True  True  True
  True  True  True  True]


41693

In [215]:
for key in proclus_h4: clusters_info(proclus_h4[key])

Scheme: PROCLUS K=4;D=50
 Number of Clusters 4
Average dimensions = 50.0
----------------------------------------------------------------------------------------------------
Scheme: PROCLUS K=4;D=75
 Number of Clusters 4
Average dimensions = 75.0
----------------------------------------------------------------------------------------------------
Scheme: PROCLUS K=4;D=95
 Number of Clusters 4
Average dimensions = 95.0
----------------------------------------------------------------------------------------------------


In [218]:
sample_sentences_from_cluster_sub(proclus_h4['exp_k4_d95'], 2, 10, halo_4)

[['Sentence #153:',
  'A2QMWLLWU52N7J|B0050SYX8W',
  "awsome great game , love it awsome what can i say ? it 's halo ... ! ! ! ! awsome great game , love it awsome what can i say ? it 's halo ... ! ! ! !"],
 ['Sentence #278:',
  'A2ADPMDVK2C5JU|B0050SYX8W',
  "i will be brief.great campaign , wicked new promethean enemies , smooth solid online multi-player . i do n't need to say much else about it at this point , everyone else has already enlightened you as to what you are in for.halo 4 is a fantastic game through and through , you really need to check it out and see if it fits your tastes or not . if your on the fence at the very least rent it , but i can safely recommend this to almost anyone who likes the fps genre and has a live account ."],
 ['Sentence #297:',
  'A2P4QIW2G6U8YZ|B0050SYX8W',
  'this is an awesome game , i finally was able to view a video about from where halo originated , so now i know who is master chief and the hole game makes a lot of sense , great graphics'],
 

In [162]:
ALPHA = [0.01]#, 0.1]
BETA = [0.2, 0.4]
MAXOUT = [1]
K = [20] #up to 64?
numBins = [1]
W = [1.1]
mineclus_h4 = {}
Mineclus_Experiment(ALPHA, BETA, MAXOUT, K, numBins, W, mineclus_h4, 'halo_4.arff')

Done MineClus_a0.01_b0.2_m1_k20_n1_w1.1  in  14.447848558425903  sec
Done MineClus_a0.01_b0.4_m1_k20_n1_w1.1  in  12.45397138595581  sec


In [163]:
clusters_info(mineclus_h4['exp_a0.01_b0.2_m1_k20_n1_w1.1'])
clusters_info(mineclus_h4['exp_a0.01_b0.4_m1_k20_n1_w1.1'])
print(mineclus_h4['exp_a0.01_b0.2_m1_k20_n1_w1.1'][85])
print(mineclus_h4['exp_a0.01_b0.4_m1_k20_n1_w1.1'][70])

0
 Number of Clusters 101
Average dimensions = 97.3
----------------------------------------------------------------------------------------------------
0
 Number of Clusters 86
Average dimensions = 98.0
----------------------------------------------------------------------------------------------------
SC_2: [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 ] #47 {38 97 164 173 41 136 218 259 347 371 378 385 40 125 252 258 321 59 67 156 311 319 346 76 223 370 36 242 368 51 106 209 122 180 296 60 166 205 75 264 309 139 225 304 215 248 288 }

SC_2: [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 ] #9 {72 138 157 193 220 225 35 199 372 }



In [216]:
pp = pprint.PrettyPrinter(width=120)

print(mineclus_h4['exp_a0.01_b0.2_m1_k20_n1_w1.1'][83])
pp.pprint(sample_sentences_from_cluster_sub(mineclus_h4['exp_a0.01_b0.2_m1_k20_n1_w1.1'], 83, 5, halo_4))

print(mineclus_h4['exp_a0.01_b0.2_m1_k20_n1_w1.1'][84])
pp.pprint(sample_sentences_from_cluster_sub(mineclus_h4['exp_a0.01_b0.2_m1_k20_n1_w1.1'], 84, 5, halo_4))

print(mineclus_h4['exp_a0.01_b0.2_m1_k20_n1_w1.1'][85])
pp.pprint(sample_sentences_from_cluster_sub(mineclus_h4['exp_a0.01_b0.2_m1_k20_n1_w1.1'], 85, 5, halo_4))



SC_0: [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 ] #83 {108 232 247 306 390 7 8 14 58 102 104 117 206 216 286 354 361 380 382 388 393 30 66 325 0 70 87 236 335 83 178 256 315 358 32 95 292 356 39 165 381 124 134 340 71 116 161 37 138 177 272 1 3 19 21 80 151 191 202 13 121 369 15 187 353 84 159 348 160 308 322 25 254 310 175 230 345 20 78 332 72 133 162 }

[['Sentence #25:',
  'A3SUB9214D8AAL|B0050SYX8W',
  "not like the other halo 's story is sloppy along with the game play , slow reaction time with controls , weapons in "
  'game are not that fun to use . head shots do not really matter with lower end weapons.. still takes alot of shots '
  'to get the kill .'],
 ['Sentence #247:',
  'A3NC3SGPULMY4Q|B0050SYX8W',
  "having played all the previous halo games and being impressed at what 343 industries did with the original game 's 

In [151]:
with open('cluster_halo4.json', 'w') as fp:
    json.dump(sample_sentences_from_cluster_sub(mineclus_h4['exp_a0.01_b0.2_m1_k20_n1_w1.1'], 81, 10, halo_4), fp)

with open('cluster_halo4.json') as halo4_samples:    
    halo4json = json.load(halo4_samples)

print(halo4json)

[['Sentence #1:', 'A3HKP40VGRAKJL|B0050SYX8W', "first off.i want to let you know that im not a big halo fan . i played the crap outta the first one , skipped the second one , played even more of the 3rd one as well as odst and reach . i used to be a big halo fan but it seems to me as time went on i started to be more into cod franchise . cod 4 , mw2 , and blops . last year i bought battlefield and have played that solidly for a year.ok so now you know my gaming history for the past years . lets talk about halo 4. i had a big decision this year . i went and bought premium of battlefield ( $ 50-which if youre thinking about picking it up , just buy the 'premium edition ' and you get everything included for $ 60 ) anyway my budget allowed me to get one more game for this year , and it boiled down to halo 4 or blops 2. as some of you could guess by now , halo 4 was my choice . and i am very happy with my decision.i knew blops would disappoint me , especially coming from a such a big fan of

In [224]:
print(proclus_h4['exp_k4_d95'][2])
pp.pprint(sample_sentences_from_cluster_sub(proclus_h4['exp_k4_d95'], 2, 15, halo_4))

print(proclus_h4['exp_k4_d95'][3])
pp.pprint(sample_sentences_from_cluster_sub(proclus_h4['exp_k4_d95'], 3, 15, halo_4))

print(proclus_h4['exp_k4_d95'][4])
pp.pprint(sample_sentences_from_cluster_sub(proclus_h4['exp_k4_d95'], 4, 15, halo_4))

print(proclus_h4['exp_k4_d95'][5])
pp.pprint(sample_sentences_from_cluster_sub(proclus_h4['exp_k4_d95'], 5, 15, halo_4))

SC_0: [1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 ] #111 {0 2 3 5 7 11 15 16 20 21 28 29 31 65 66 70 73 79 80 82 83 85 91 92 94 96 97 102 103 104 106 107 117 123 124 126 128 129 133 134 136 137 141 142 145 146 150 151 153 156 165 167 168 172 173 176 178 182 185 191 192 193 197 200 203 206 210 216 227 228 232 236 243 252 253 258 259 263 270 278 285 286 290 292 295 297 298 307 315 317 318 323 325 326 327 328 329 352 354 355 357 358 360 363 367 371 373 380 390 391 395 }

[['Sentence #20:',
  'AFNG8O2DXRCUV|B0050SYX8W',
  'while our family plays kinect games , i personally play action / role-playing games on ps3 . i loved the ps3 '
  'exclusives like uncharted , killzone , resistance , but also loved cross-platform games like cod series , '
  'battlefield , and especially mass effect 2 and 3. i love the strong stories of mass effect ba